### This notebook checks the QE curves as they are in obs_lsst_data repo

This is a modified version of the notebook by Simon Krughoff
https://github.com/lsst-sitcom/notebooks/blob/master/contrib/get_QE_curve.ipynb

Simon's notebook looks at all the QE curves that are in the ts8/CALIB. 
Here we specify a raft name (e.g. R22), then only plot the 9 CCDs for that raft.

Other added features:
* compare to the design curve in syseng_throughput, and 
* plot the raw data points so that we can examine the interpolation/extrapolation. 
 - We don't want to make the plots too busy so we only show raw data points averaged over the amps.

In [3]:
# make sure this is w_2019_49 or later
! eups list | grep lsst_distrib 

ERROR: ld.so: object '/etc/alternatives/java_sdk_openjdk/jre/lib/amd64/libjsig.so' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/etc/alternatives/java_sdk_openjdk/jre/lib/amd64/libjsig.so' from LD_PRELOAD cannot be preloaded: ignored.
ERROR: ld.so: object '/etc/alternatives/java_sdk_openjdk/jre/lib/amd64/libjsig.so' from LD_PRELOAD cannot be preloaded: ignored.
lsst_distrib          19.0.0+4   	w_2019_49 current setup


In [4]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, export_png
output_notebook()

from lsst.daf.persistence import Butler, NoResults
import os, numpy
import pandas as pd

Loading BokehJS ...

In [27]:
import lsst.syseng.throughputs as st
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
from lsst.obs.lsst.lsstCamMapper import LsstCamMapper

In [6]:
rname = 'R10'

In [7]:
mapper = LsstCamMapper()
cam = mapper.camera

In [8]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/ts8/CALIB"
print(DATADIR)
butler = Butler(DATADIR)
ts8cam = butler.get('camera')

/home/bxin/lsst_stack/obs_lsst/ts8/CALIB


In [70]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
rtmname = dd.rtm.loc[rname]
vendor = dd.vendor.loc[rname]
print(rname, rtmname,vendor)

R10 RTM-023 ITL


### Retrieve the vendor design curve (e2v or ITL)

In [78]:
defaultDirs = st.setDefaultDirs()
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector = st.buildDetector(vendorDir, addLosses)

In [89]:
def make_subplot(plot, n):
    xmin = None
    xmax = None
    interp_list = []
    for k in qe_curve.data:
        wlen = qe_curve.data[k][0]
        #print(wlen)
        eff = qe_curve.data[k][1]    
        plot.circle(wlen.value, eff.value, line_color='black', alpha=0.1, line_width=3)
        
        amp_point = det[k].getBBox().getCenter()
        #print(det[k].getName())
        if xmin is None or xmax is None:
            xmin = wlen.min()
            xmax = wlen.max()
            interp_wlen = numpy.linspace(xmin, xmax, num=500)
        interp_eff = qe_curve.evaluate(det, amp_point, interp_wlen, kind='quadratic')
        #print(interp_wlen)
        interp_list.append(interp_eff)
        
    interp_arr = numpy.array([arr for arr in interp_list])
    median_eff = numpy.median(interp_arr, axis=0)
    #print(interp_arr.shape)
    plot.line(interp_wlen, median_eff, color='black', line_width=2)
    plot.line(detector.wavelen, detector.sb*100, color='red', line_width=2)
    
    if n > 6:
        plot.xaxis.axis_label = 'wavelength (nm)'
        
    if n in (1, 4, 7):
        plot.yaxis.axis_label = 'efficiency (%)'
        
    plot.text(x=[400,], y=[20,], text=[f'Detector: {det.getName()}',])

In [90]:
plots = []
for det in ts8cam:
    rtmname1, dname = det.getName().split('_')
    if rtmname1 != rtmname:
        continue;

    try:
        qe_curve = butler.get('qe_curve', raftName=rtmname, detectorName=dname, taiObs='2000-01-01T00:00:00')
    except NoResults:
        continue  # No results found for this detector

    if len(plots) > 0:
        plots.append(figure(x_range=plots[0].x_range, y_range=plots[0].y_range))
    else:
        plots.append(figure())
    print(rtmname1, rtmname)
    make_subplot(plots[-1], len(plots))

RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023
RTM-023 RTM-023


In [91]:
grid = gridplot(plots, ncols=3, plot_width=350, plot_height=200)
show(grid)